In [ ]:
from ultralytics import YOLO, SAM
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import utils
import math
import os
import json
import cv2
import glob

In [ ]:
model = YOLO("best.pt")

In [ ]:

path = "datasets/zweite_messung"
results = model.predict(path)
lst = os.listdir(path)
data = {}

In [ ]:
def area(mask, angle, distance):
    angleFactor = 1/math.cos(angle*math.pi/180)
    distanceFactor = distance*distance
    correctedHeight = int(mask.shape[0]*angleFactor)
    resized = cv2.resize(mask, dsize=(mask.shape[1], correctedHeight), interpolation=cv2.INTER_CUBIC)
    print("angleFactor", angleFactor, "correctedHeight", correctedHeight, "distanceFactor", distanceFactor)
    return np.sum(resized)* distanceFactor

In [ ]:
for f in lst:
    _, name  = os.path.split(f)
    base, ending = name.split(".")
    number = base.split("_")[0]
    if ending != "json":
        continue
    with open(os.path.join(path, f))as file:
        d = json.load(file)
        data[number] = d

yolodata = {}

for result in results:
    biggest = 0
    biggest_a = 0
    for index, (_, _, w, h) in enumerate(result.boxes.xywh):
        a = w*h
        if a > biggest_a:
            biggest = index
            biggest_a = a

    _, name = os.path.split(result.path)
    number = name.split("_")[0]
    mask = result.masks.data.cpu().numpy()[biggest, :, :]
    angle = data[number]["Angle"]
    distance = data[number]["Distance"]
    proxy = area(mask, angle, distance)
    conf = result.boxes.conf.cpu().numpy()[biggest]
    print(name)
    print("raw sum", np.sum(mask))
    print("index", biggest, "conf", conf, "proxy", proxy, "angle", angle, "distance", distance)
    utils.plot(result)
    yolodata[number] = proxy

In [ ]:
def distance_from_name(name):
    if "Diego" in name:
        return 2.3
    if "Twentyone" in name:
        return 1.64
    if "Carina" in name:
        return 1.59
    if "Zara" in name:
        return 1.62
    if "Zarra" in name:
        return 1.62
    if "36175" in name:
        return 1.27
    raise Exception("unknown name", name)

In [ ]:
path = "datasets/erste_messung"
results = model.predict(path)
lst = os.listdir(path)
data = {}

In [ ]:
for result in results:
    head, tail = os.path.split(result.path)
    mask = result.masks.data.cpu().numpy().squeeze()
    angle = 20
    distance = distance_from_name(tail)
    utils.plot(result)
    print(tail)
    print(area(mask, angle, distance))

In [ ]:
path = "datasets/zweite_messung"
model = SAM("sam2.1_b.pt")

samdata = {}
for img_path in glob.glob("datasets/referenz/*.png"):
    _, tail = os.path.split(img_path)
    number = tail.split("_")[0]
    # Run inference with bboxes prompt
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    print(height, width)
    results = model(img, points=[width/2, height/2])
    angle = data[number]["Angle"]
    distance = data[number]["Distance"]
    mask = results[0].masks.data.cpu().numpy().squeeze().astype(float)
    proxy = area(mask, angle, distance)
    print(tail)
    print(proxy)
    samdata[number] = proxy
    utils.plot(results[0])

In [ ]:
for key in yolodata.keys():
    yp = yolodata[key]
    sp = samdata[key]
    print(key)
    print("yolo", yp)
    print("sam", sp)
    print("yp/sp", yp/sp)
    print()

In [ ]:
path = "datasets/erste_referenz"
model = YOLO("best.pt")
results = model.predict(path)

for result in results:
    _, name = os.path.split(result.path)
    number = name.split("_")[0]
    mask = result.masks.data.cpu().numpy().squeeze()
    angle = 20
    distance = distance_from_name(name)
    proxy = area(mask, angle, distance)
    print(name)
    print("proxy", proxy)
    utils.plot(result)

In [ ]:
# From what I learned by taking calibration photos 
# at 20cm 1cm is equal to 31.73 pixels
# at 30cm 1cm is equal to 22.6190476190 pixels
# at 2m 1cm is equal to 2.835 pixels
normalized_area1 = 22.6190476190*22.6190476190*0.3*0.3
normalized_area2 = 31.73*31.73*0.2*0.2
normalized_area3 = 2.835*2.835*2*2
print(normalized_area1)
print(normalized_area2)
print(normalized_area3)

In [ ]:
# pearson cooefficient for the area for erste messung
# there is no correction for the angle done since data was not available
from scipy.stats import pearsonr
weights = [65.9,82.3,60,74.5,34.8,
           67.3,71.6,58,39.4,72.5,37.1,30.7,38.6]
lengths = [85,88,85,94,70,
          83,89,81,71,88,67,65,67,]
hg = [98,105,92,104,79,
      98,97,92,80,101,78,80,81]
sacrum = [73,75,72,77,63,
         74,74,72,69,77,67,62,67,]
shoulder = [74,75,72,79,62,
            70,71,69,65,79,63,60,65]
areas = [126217.6013,157085.518,146242.3731,192738.8976,95095.41843]
#area_result = pearsonr(areas, weights)
#print("area","r", area_result.correlation, "p", area_result.pvalue)
hg_result = pearsonr(hg, weights)
print("hg","r", hg_result.correlation, "p", hg_result.pvalue)
shoulder_result = pearsonr(shoulder, weights)
print("shoulder", "r", shoulder_result.correlation, "p", shoulder_result.pvalue)
sacrum_result = pearsonr(sacrum, weights)
print("sacrum", "r", sacrum_result.correlation, "p", sacrum_result.pvalue)
length_result = pearsonr(lengths, weights)
print("length", "r", length_result.correlation, "p", length_result.pvalue)